In [1]:
#https://github.com/yufengg/widendeep/blob/master/wnd_criteo.ipynb
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time

import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO) # Set to INFO for tracking training, default is WARN. ERROR for least messages

print("Using TensorFlow version %s\n" % (tf.__version__))


CONTINUOUS_COLUMNS =  ["I"+str(i) for i in range(1,14)] # 1-13 inclusive
CATEGORICAL_COLUMNS = ["C"+str(i) for i in range(1,27)] # 1-26 inclusive
LABEL_COLUMN = ["clicked"]

TRAIN_DATA_COLUMNS = LABEL_COLUMN + CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS
# TEST_DATA_COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS

FEATURE_COLUMNS = CONTINUOUS_COLUMNS + CATEGORICAL_COLUMNS

print('Feature columns are: ', FEATURE_COLUMNS, '\n')

 # label is 1
sample = [ 0 , 2, 11, 5, 10262, 34, 2, 4, 5,0 , 1,0 , 5, "be589b51", "287130e0", "cd7a7a22", "fb7334df", "25c83c98","0" , "6cdb3998", "361384ce", "a73ee510", "3ff10fb2", "5874c9c9", "976cbd4c", "740c210d", "1adce6ef", "310d155b", "07eb8110", "07c540c4", "891589e7", "18259a83", "a458ea53", "a0ab60ca","0" , "32c7478e", "a052b1ed", "9b3e8820", "8967c0d2"]

# label is 1
sample = [ 0, 127, 1, 3, 1683, 19, 26, 17, 475, 0, 9, 0, 3, "05db9164", "8947f767", "11c9d79e", "52a787c8", "4cf72387", "fbad5c96", "18671b18", "0b153874", "a73ee510", "ceb10289", "77212bd7", "79507c6b", "7203f04e", "07d13a8f", "2c14c412", "49013ffe", "8efede7f", "bd17c3da", "f6a3e43b", "a458ea53", "35cd95c9", "ad3062eb", "c7dc6720", "3fdb382b", "010f6491", "49d68486"]

print('Columns and data as a dict: ', dict(zip(FEATURE_COLUMNS, sample)), '\n')

Using TensorFlow version 1.10.0

Feature columns are:  ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'] 

Columns and data as a dict:  {'I1': 0, 'I2': 127, 'I3': 1, 'I4': 3, 'I5': 1683, 'I6': 19, 'I7': 26, 'I8': 17, 'I9': 475, 'I10': 0, 'I11': 9, 'I12': 0, 'I13': 3, 'C1': '05db9164', 'C2': '8947f767', 'C3': '11c9d79e', 'C4': '52a787c8', 'C5': '4cf72387', 'C6': 'fbad5c96', 'C7': '18671b18', 'C8': '0b153874', 'C9': 'a73ee510', 'C10': 'ceb10289', 'C11': '77212bd7', 'C12': '79507c6b', 'C13': '7203f04e', 'C14': '07d13a8f', 'C15': '2c14c412', 'C16': '49013ffe', 'C17': '8efede7f', 'C18': 'bd17c3da', 'C19': 'f6a3e43b', 'C20': 'a458ea53', 'C21': '35cd95c9', 'C22': 'ad3062eb', 'C23': 'c7dc6720', 'C24': '3fdb382b', 'C25': '010f6491', 'C26': '49d68486'} 



In [4]:
BATCH_SIZE = 400
def generate_input_fn(filename, batch_size=BATCH_SIZE):
    def _input_fn():
        filename_queue = tf.train.string_input_producer([filename])
        reader = tf.TextLineReader()
        # Reads out batch_size number of lines
        key, value = reader.read_up_to(filename_queue, num_records=batch_size)
        
        # 1 int label, 13 ints, 26 strings
        cont_defaults = [ [0] for i in range(1,14) ]
        cate_defaults = [ [" "] for i in range(1,27) ]
        label_defaults = [ [0] ]
        column_headers = TRAIN_DATA_COLUMNS
        # The label is the first column of the data.
        record_defaults = label_defaults + cont_defaults + cate_defaults

        # Decode CSV data that was just read out. 
        # Note that this does NOT return a dict, 
        # so we will need to zip it up with our headers
        columns = tf.decode_csv(
            value, record_defaults=record_defaults)
        
        # all_columns is a dictionary that maps from column names to tensors of the data.
        all_columns = dict(zip(column_headers, columns))
        
        # Pop and save our labels 
        # dict.pop() returns the popped array of values; exactly what we need!
        labels = all_columns.pop(LABEL_COLUMN[0])
        
        # the remaining columns are our features
        features = all_columns 

        # Sparse categorical features must be represented with an additional dimension. 
        # There is no additional work needed for the Continuous columns; they are the unaltered columns.
        # See docs for tf.SparseTensor for more info
        for feature_name in CATEGORICAL_COLUMNS:
            features[feature_name] = tf.expand_dims(features[feature_name], -1)

        return features, labels

    return _input_fn

print('input function configured')

input function configured


In [5]:
# Sparse base columns.
# C1 = tf.contrib.layers.sparse_column_with_hash_bucket('C1', hash_bucket_size=1000)
# C2 = tf.contrib.layers.sparse_column_with_hash_bucket('C2', hash_bucket_size=1000)
# C3 = tf.contrib.layers.sparse_column_with_hash_bucket('C3', hash_bucket_size=1000)
# ...
# Cn = tf.contrib.layers.sparse_column_with_hash_bucket('Cn', hash_bucket_size=1000)
# wide_columns = [C1, C2, C3, ... , Cn]

wide_columns = []
for name in CATEGORICAL_COLUMNS:
    wide_columns.append(tf.contrib.layers.sparse_column_with_hash_bucket(
            name, hash_bucket_size=1000))

print('Wide/Sparse columns configured')

Wide/Sparse columns configured


In [6]:
# Continuous base columns.
# I1 = tf.contrib.layers.real_valued_column("I1")
# I2 = tf.contrib.layers.real_valued_column("I2")
# I3 = tf.contrib.layers.real_valued_column("I3")
# ...
# In = tf.contrib.layers.real_valued_column("In")
# deep_columns = [I1, I2, I3, ... , In]

deep_columns = []
for name in CONTINUOUS_COLUMNS:
    deep_columns.append(tf.contrib.layers.real_valued_column(name))

print('deep/continuous columns configured')

deep/continuous columns configured


In [7]:
# Examples from other datasets are shown below.

# age_buckets = tf.contrib.layers.bucketized_column(age,
#             boundaries=[ 18, 25, 30, 35, 40, 45, 50, 55, 60, 65 ])
# education_occupation = tf.contrib.layers.crossed_column([education, occupation], 
#                                                         hash_bucket_size=int(1e4))
# age_race_occupation = tf.contrib.layers.crossed_column([age_buckets, race, occupation], 
#                                                        hash_bucket_size=int(1e6))
# country_occupation = tf.contrib.layers.crossed_column([native_country, occupation], 
#                                                       hash_bucket_size=int(1e4))

print('Transformations complete')

Transformations complete


In [8]:
# Embeddings for wide columns into deep columns
for col in wide_columns:
    deep_columns.append(tf.contrib.layers.embedding_column(col, 
                                                           dimension=8))

print('wide and deep columns configured')

wide and deep columns configured


In [9]:
def create_model_dir(model_type):
    # Returns something like models/model_WIDE_AND_DEEP_1493043407
    return 'models/model_' + model_type + '_' + str(int(time.time()))

# Specify the desired model_dir 
def get_model(model_type, model_dir):
    print("Model directory = %s" % model_dir)
    
    # There are more options here than shown here. 
    # We are using this to show additional checkpointing for illustrative purposes.
    # In a real system with far more samples, you would 
    #     likely choose to save checkpoints less frequently.
    runconfig = tf.contrib.learn.RunConfig(
        save_checkpoints_secs=None,
        save_checkpoints_steps = 100,
    )
    
    m = None
    
    # Linear Classifier
    if model_type == 'WIDE':
        m = tf.contrib.learn.LinearClassifier(
            model_dir=model_dir, 
            feature_columns=wide_columns)

    # Deep Neural Net Classifier
    if model_type == 'DEEP':
        m = tf.contrib.learn.DNNClassifier(
            model_dir=model_dir,
            feature_columns=deep_columns,
            hidden_units=[100, 50, 25])

    # Combined Linear and Deep Classifier
    if model_type == 'WIDE_AND_DEEP':
            m = tf.contrib.learn.DNNLinearCombinedClassifier(
            model_dir=model_dir,
            linear_feature_columns=wide_columns,
            dnn_feature_columns=deep_columns,
            dnn_hidden_units=[100, 70, 50, 25],
            config=runconfig)
        
    print('estimator built')
    
    return m
    

MODEL_TYPE = 'WIDE_AND_DEEP'
model_dir = create_model_dir(model_type=MODEL_TYPE)
m = get_model(model_type=MODEL_TYPE, model_dir=model_dir)

Model directory = models/model_WIDE_AND_DEEP_1602323617
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
Instructions for updating:
Please set fix_global_step_increment_bug=True and update training steps in your pipeline. See pydoc for details.
Instructions for updating:
Please switch to tf.contrib.estimator.*_head.
Instructions for updating:
Please replace uses of any Estimator from tf.contrib.learn with an Estimator from tf.estimator.*
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1201c9b38>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_

In [10]:
from tensorflow.contrib.learn.python.learn import evaluable
isinstance(m, evaluable.Evaluable)

True

In [12]:
# Use the cloud or local depending on your preference

train_file = "./train.csv"
eval_file  = "./eval.csv"

train_sample_size = 800000
train_steps = train_sample_size/BATCH_SIZE # 8000/40 = 200

m.fit(input_fn=generate_input_fn(train_file, BATCH_SIZE), steps=train_steps)

print('fit done')

eval_sample_size = 200000 # this can be found with a 'wc -l eval.csv'
eval_steps = eval_sample_size/BATCH_SIZE # 2000/40 = 50

results = m.evaluate(input_fn=generate_input_fn(eval_file), 
                     steps=eval_steps)
print('evaluate done')

print('Accuracy: %s' % results['accuracy'])
print(results)

Instructions for updating:
Use the `axis` argument instead
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.EstimatorSpec. You can use the `estimator_spec` method to create an equivalent one.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into models/model_WIDE_AND_DEEP_1602323617/model.ckpt.
INFO:tensorflow:loss = 83.7912, step = 2
INFO:tensorflow:Saving checkpoints for 102 into models/model_WIDE_AND_DEEP_1602323617/model.ckpt.
INFO:tensorflow:global_step/sec: 10.0802
INFO:tensorflow:loss = 0.8525928, step = 202 (15.130 sec)
INFO:tensorflow:Saving checkpoints for 204 into models/model_WIDE_AND_DEEP_1602323617/model.ckpt.
INFO:tensorflow:global_step/sec: 18.0884
INFO:tensorflow:Saving checkpoints for 306 into models/model_WIDE_AND_DEEP_1602323617/model.ckpt.
INFO:tensorflow:global_step/sec: 69.

INFO:tensorflow:Starting evaluation at 2020-10-10-09:55:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1602323617/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [50/500]
INFO:tensorflow:Evaluation [100/500]
INFO:tensorflow:Evaluation [150/500]
INFO:tensorflow:Evaluation [200/500]
INFO:tensorflow:Evaluation [250/500]
INFO:tensorflow:Evaluation [300/500]
INFO:tensorflow:Evaluation [350/500]
INFO:tensorflow:Evaluation [400/500]
INFO:tensorflow:Evaluation [450/500]
INFO:tensorflow:Evaluation [500/500]
INFO:tensorflow:Finished evaluation at 2020-10-10-09:56:08
INFO:tensorflow:Saving dict for global step 2002: accuracy = 0.782, accuracy/baseline_label_mean = 0.209, accuracy/threshold_0.500000_mean = 0.782, auc = 0.713017, auc_precision_recall = 0.37578017, global_step = 2002, labels/actual_label_mean = 0.209, labels/prediction_mean = 0.21569468, loss = 

In [17]:
def pred_fn():
    sample = [ 0, 127, 1, 3, 1683, 19, 26, 17, 475, 0, 9, 0, 3, "05db9164", "8947f767", "11c9d79e", "52a787c8", "4cf72387", "fbad5c96", "18671b18", "0b153874", "a73ee510", "ceb10289", "77212bd7", "79507c6b", "7203f04e", "07d13a8f", "2c14c412", "49013ffe", "8efede7f", "bd17c3da", "f6a3e43b", "a458ea53", "35cd95c9", "ad3062eb", "c7dc6720", "3fdb382b", "010f6491", "49d68486"]
    sample_dict = dict(zip(FEATURE_COLUMNS, sample))
    
    for feature_name in CATEGORICAL_COLUMNS:
        sample_dict[feature_name] = tf.expand_dims(sample_dict[feature_name], -1)
        
    for feature_name in CONTINUOUS_COLUMNS:
        sample_dict[feature_name] = tf.expand_dims(tf.constant(sample_dict[feature_name], dtype=tf.int32), -1)

    print(sample_dict)

    return sample_dict

m.predict(input_fn=pred_fn)

{'I1': <tf.Tensor 'ExpandDims_26:0' shape=(1,) dtype=int32>, 'I2': <tf.Tensor 'ExpandDims_27:0' shape=(1,) dtype=int32>, 'I3': <tf.Tensor 'ExpandDims_28:0' shape=(1,) dtype=int32>, 'I4': <tf.Tensor 'ExpandDims_29:0' shape=(1,) dtype=int32>, 'I5': <tf.Tensor 'ExpandDims_30:0' shape=(1,) dtype=int32>, 'I6': <tf.Tensor 'ExpandDims_31:0' shape=(1,) dtype=int32>, 'I7': <tf.Tensor 'ExpandDims_32:0' shape=(1,) dtype=int32>, 'I8': <tf.Tensor 'ExpandDims_33:0' shape=(1,) dtype=int32>, 'I9': <tf.Tensor 'ExpandDims_34:0' shape=(1,) dtype=int32>, 'I10': <tf.Tensor 'ExpandDims_35:0' shape=(1,) dtype=int32>, 'I11': <tf.Tensor 'ExpandDims_36:0' shape=(1,) dtype=int32>, 'I12': <tf.Tensor 'ExpandDims_37:0' shape=(1,) dtype=int32>, 'I13': <tf.Tensor 'ExpandDims_38:0' shape=(1,) dtype=int32>, 'C1': <tf.Tensor 'ExpandDims:0' shape=(1,) dtype=string>, 'C2': <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=string>, 'C3': <tf.Tensor 'ExpandDims_2:0' shape=(1,) dtype=string>, 'C4': <tf.Tensor 'ExpandDims_3:0' sha

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1602323617/model.ckpt-2002
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


<generator object _as_iterable at 0x127e02a40>

In [21]:
from tensorflow.contrib.learn.python.learn.utils import input_fn_utils

def column_to_dtype(column):
    if column in CATEGORICAL_COLUMNS:
        return tf.string
    else:
        return tf.float32

def serving_input_fn():
    feature_placeholders = {
        column: tf.placeholder(column_to_dtype(column), [None])
        for column in FEATURE_COLUMNS
    }
    # DNNCombinedLinearClassifier expects rank 2 Tensors, but inputs should be
    # rank 1, so that we can provide scalars to the server
    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in feature_placeholders.items()
    }
    
    return input_fn_utils.InputFnOps(
        features, # input into graph
        None,
        feature_placeholders # tensor input converted from request 
    )
export_folder = m.export_savedmodel(
    model_dir + '/export',
    serving_input_fn
)

print('model exported successfully to {}'.format(export_folder))

Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
INFO:tensorflow:Restoring parameters from models/model_WIDE_AND_DEEP_1602323617/model.ckpt-2002
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models/model_WIDE_AND_DEEP_1602323617/export/temp-1602324046/saved_model.pb
model exported successfully to b'models/model_WIDE_AND_DEEP_1602323617/export/1602324046'


In [22]:
from tensorflow.contrib.learn.python.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import saved_model_export_utils


# get_model(model_type = 'WIDE_AND_DEEP', model_dir=model_dir)

# output_dir is an arg passed in by the learn_runner.run() call.
def experiment_fn(output_dir):
    
    print(output_dir)
    
    train_input_fn = generate_input_fn(train_file, BATCH_SIZE)
    eval_input_fn = generate_input_fn(eval_file)
    my_model = get_model(model_type=MODEL_TYPE, 
                  model_dir=output_dir)

    experiment = tf.contrib.learn.Experiment(
        my_model,
        train_input_fn=train_input_fn,
        eval_input_fn=eval_input_fn,
        train_steps=1000
        ,
        export_strategies=[saved_model_export_utils.make_export_strategy(
            serving_input_fn,
            default_output_alternative_key=None,
            exports_to_keep=1
        )]
    )
    return experiment
exp = experiment_fn(model_dir)

exp.train_and_evaluate()

models/model_WIDE_AND_DEEP_1602323617
Model directory = models/model_WIDE_AND_DEEP_1602323617
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12fa294e0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'models/model_WIDE_AND_DEEP_1602323617'}
estimator built
Instructions for updating:
Switch to tf.estimator.Exporter and associated utilities.
Instructions for updating:
Please switch to tf.estimator.train_and_evaluate, and use tf.estimator.E

({'loss': 0.4710025,
  'accuracy': 0.782,
  'labels/prediction_mean': 0.21569496,
  'labels/actual_label_mean': 0.209,
  'accuracy/baseline_label_mean': 0.209,
  'auc': 0.713017,
  'auc_precision_recall': 0.37578017,
  'accuracy/threshold_0.500000_mean': 0.782,
  'precision/positive_threshold_0.500000_mean': 0.4296875,
  'recall/positive_threshold_0.500000_mean': 0.13157895,
  'global_step': 2002},
 [b'models/model_WIDE_AND_DEEP_1602323617/export/Servo/1602324110'])

In [ ]:
model_dir=create_model_dir(model_type=MODEL_TYPE)
metrics, output_folder = learn_runner.run(experiment_fn, model_dir)

print('Accuracy: {}'.format(metrics['accuracy']))
print('Model exported to {}'.format(output_folder))

Instructions for updating:
Use tf.estimator.train_and_evaluate.
models/model_WIDE_AND_DEEP_1602324131
Model directory = models/model_WIDE_AND_DEEP_1602324131
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1201b1b38>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': 100, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'models/model_WIDE_AND_DEEP_1602324131'}
estimator built
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
